<a href="https://colab.research.google.com/github/zawadalam7/Zawad_DTSC3020_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [1]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [15]:
import re
import csv
from pathlib import Path

input_path = Path("contacts_raw.txt")
output_path = Path("contacts_clean.csv")

email_regex = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

def normalize_phone(raw):
  digits = re.sub(r"\D", "", raw)
  return digits[-10:] if len(digits) >= 10 else ""

try:
  with input_path.open("r", encoding="utf-8") as f:
    lines = f.readlines()
except FileNotFoundError:
  print("contacts_raw.txt not found. Please check the file path.")
else:
  seen_emails = set()
  cleaned_rows = []

  for line in lines:
    parts = [p.strip() for p in line.split(",")]
    if len(parts) < 2:
      continue

    name_email = parts[0]
    phone_raw = parts[1] if len(parts) > 1 else ""

    email_match = re.search(r"<([^>]+)>", name_email)
    if email_match:
      email = email_match.group(1).strip()
      name = re.sub(r"<[^>]*>", "", name_email).strip()
    else:
      if len(parts) >= 3:
        name = parts[0]
        email = parts[1]
        phone_raw = parts[2]
      else:
        continue

    if not email_regex.fullmatch(email.strip()):
      continue

    email_key = email.casefold()
    if email_key in seen_emails:
      continue
    seen_emails.add(email_key)

    phone = normalize_phone(phone_raw)

    cleaned_rows.append([name, email, phone])

  with output_path.open("w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "email", "phone"])
    writer.writerows(cleaned_rows)

## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [19]:
import unittest
import re

email_regex = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")

def validate_email(email):
  return bool(email_regex.fullmatch(email.strip()))

def normalize_phone(raw):
  digits = re.sub(r"\D", "", raw)
  return digits[-10:] if len(digits) >= 10 else ""

def parse_contacts(lines):
  seen = set()
  cleaned = []

  for line in lines:
    parts = [p.strip() for p in line.split(",")]
    if len(parts) < 2:
      continue

    name_email = parts[0]
    phone_raw = parts[1] if len(parts) > 1 else ""

    email_match = re.search(r"<([^>]+)>", name_email)
    if email_match:
      email = email_match.group(1).strip()
      name = re.sub(r"<[^>]*>", "", name_email).strip()
    else:
      if len(parts) >= 3:
        name = parts[0]
        email = parts[1]
        phone_raw = parts[2]
      else:
        continue

    if not validate_email(email):
      continue

    email_key = email.casefold()
    if email_key in seen:
      continue
    seen.add(email_key)

    phone = normalize_phone(phone_raw)
    cleaned.append([name, email, phone])

  return cleaned

class TestCRMCleanup(unittest.TestCase):

  def test_email_validation(self):
    self.assertTrue(validate_email("alice@example.com"))
    self.assertTrue(validate_email("bob.smith+tag@sub.domain.co.uk"))
    self.assertFalse(validate_email("bob[at]example.com"))
    self.assertFalse(validate_email("noatsign.com"))
    self.assertFalse(validate_email("missing@tld"))

  def test_phone_normalization(self):
    self.assertEqual(normalize_phone("+1 (469) 555-1234"), "4695551234")
    self.assertEqual(normalize_phone("972-555-7777"), "9725557777")
    self.assertEqual(normalize_phone("214 555 8888"), "2145558888")
    self.assertEqual(normalize_phone("972.777.2121"), "9727772121")
    self.assertEqual(normalize_phone("12345"), "")

  def test_parsing_and_structure(self):
    raw_data = """Alice Johnson <alice@example.com> , +1 (469) 555-1234
Bob <bob[at]example.com> , 972-555-7777
Sara M. , sara@mail.co , 214 555 8888
"Mehdi A." <mehdi.ay@example.org> , (469)555-9999
duplicate <ALICE@EXAMPLE.COM> , 469 555 1234"""
    expected = [
        ["Alice Johnson", "alice@example.com", "4695551234"],
        ["Sara M.", "sara@mail.co", "2145558888"],
        ["Mehdi A.", "mehdi.ay@example.org", "4695559999"]
    ]
    result = parse_contacts(raw_data.splitlines())
    self.assertEqual(result, expected)

  def test_deduplication_case_insensitive(self):
    raw_data = """A <a@example.com> , 123-456-7890
B <A@EXAMPLE.COM> , 000-000-0000
C <c@example.com> , 999-999-9999"""
    result = parse_contacts(raw_data.splitlines())
    emails = [row[1] for row in result]
    self.assertEqual(emails, ["a@example.com", "c@example.com"])

unittest.main(argv=[''], exit=False)

..F.
FAIL: test_parsing_and_structure (__main__.TestCRMCleanup.test_parsing_and_structure)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-354337241.py", line 78, in test_parsing_and_structure
    self.assertEqual(result, expected)
AssertionError: Lists differ: [['Al[83 chars]558888'], ['"Mehdi A."', 'mehdi.ay@example.org', '4695559999']] != [['Al[83 chars]558888'], ['Mehdi A.', 'mehdi.ay@example.org', '4695559999']]

First differing element 2:
['"Mehdi A."', 'mehdi.ay@example.org', '4695559999']
['Mehdi A.', 'mehdi.ay@example.org', '4695559999']

  [['Alice Johnson', 'alice@example.com', '4695551234'],
   ['Sara M.', 'sara@mail.co', '2145558888'],
-  ['"Mehdi A."', 'mehdi.ay@example.org', '4695559999']]
?    -        -

+  ['Mehdi A.', 'mehdi.ay@example.org', '4695559999']]

----------------------------------------------------------------------
Ran 4 tests in 0.007s

FAILED (failures=1)


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
